# 🌐 Healthcare Chatbot - Deployment

In this notebook, you'll:
1. Set up your trained chatbot for deployment
2. Launch a beautiful web interface
3. Test the deployed chatbot
4. Share your chatbot with others
5. Learn about production deployment options

Let's get your AI doctor online! 🚀🏥

## 🔧 Step 1: Deployment Setup

First, let's prepare your trained model for deployment.

In [ ]:
import sys
import os
import json
import subprocess
import time
import threading
from datetime import datetime
import webbrowser
from pathlib import Path

# Add src to path
sys.path.append('/workspace/src')

print("🔧 Deployment environment setup complete!")
print(f"📅 Deployment date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Find available trained models
def find_trained_models():
    """Find all trained models ready for deployment."""
    models_dir = '/workspace/models'
    trained_models = []
    
    if os.path.exists(models_dir):
        for item in os.listdir(models_dir):
            model_path = os.path.join(models_dir, item)
            if os.path.isdir(model_path):
                # Look for final_model subdirectory
                final_model_path = os.path.join(model_path, 'final_model')
                if os.path.exists(final_model_path):
                    # Check if it has the necessary files
                    required_files = ['config.json', 'pytorch_model.bin']
                    if any(os.path.exists(os.path.join(final_model_path, f)) for f in required_files):
                        trained_models.append({
                            'path': final_model_path,
                            'name': item,
                            'parent_dir': model_path
                        })
    
    return trained_models

print("🔍 Searching for deployable models...")
available_models = find_trained_models()

if available_models:
    print(f"🤖 Found {len(available_models)} deployable model(s):")
    for i, model_info in enumerate(available_models, 1):
        # Get training info if available
        summary_path = os.path.join(model_info['parent_dir'], 'kaggle_training_summary.json')
        if not os.path.exists(summary_path):
            summary_path = os.path.join(model_info['parent_dir'], 'training_summary.json')
        
        training_info = "No info available"
        if os.path.exists(summary_path):
            try:
                with open(summary_path, 'r') as f:
                    summary = json.load(f)
                
                config = summary.get('training_config', {})
                model_key = config.get('model_key', 'Unknown')
                samples = summary.get('dataset_stats', {}).get('total_samples', 'Unknown')
                training_info = f"{model_key}, {samples} samples"
                
            except:
                pass
        
        print(f"   {i}. {model_info['name']} ({training_info})")
        print(f"      Path: {model_info['path']}")
        
else:
    print("❌ No deployable models found.")
    print("\n🔧 SOLUTIONS:")
    print("1. Train a model first using notebook 03_Model_Training.ipynb")
    print("2. Or run: python train_kaggle_chatbot.py --kaggle_dataset_path your-dataset.csv")
    print("3. Make sure training completed successfully")

In [ ]:
# Select model for deployment
# MODIFY THIS: Choose which model to deploy
SELECTED_MODEL = None

if available_models:
    # Auto-select the first model
    SELECTED_MODEL = available_models[0]
    print(f"🤖 Auto-selected model: {SELECTED_MODEL['name']}")
    
    # Or manually select by index (uncomment and modify)
    # SELECTED_MODEL = available_models[0]  # Change index as needed

if SELECTED_MODEL:
    MODEL_PATH = SELECTED_MODEL['path']
    MODEL_NAME = SELECTED_MODEL['name']
    
    print(f"✅ Selected model for deployment:")
    print(f"   Name: {MODEL_NAME}")
    print(f"   Path: {MODEL_PATH}")
    
    # Verify model files
    model_files = os.listdir(MODEL_PATH)
    print(f"   Files: {len(model_files)} files found")
    
    # Check for key files
    key_files = ['config.json', 'tokenizer.json', 'pytorch_model.bin', 'model.safetensors']
    found_files = [f for f in key_files if f in model_files]
    print(f"   Key files: {', '.join(found_files)}")
    
    deployment_ready = True
    
else:
    print("❌ No model selected for deployment.")
    deployment_ready = False
    MODEL_PATH = None
    MODEL_NAME = None

## 🧪 Step 2: Pre-Deployment Testing

Let's quickly test your model before deploying it.

In [ ]:
# Quick model test
if deployment_ready:
    print("🧪 PRE-DEPLOYMENT TESTING")
    print("="*35)
    
    try:
        from chatbot import HealthcareChatbot
        
        print("🔄 Loading chatbot...")
        chatbot = HealthcareChatbot(MODEL_PATH)
        
        print("✅ Chatbot loaded successfully!")
        
        # Quick test questions
        test_questions = [
            "Hello!",
            "What are the symptoms of fever?",
            "What's the weather like?"
        ]
        
        print("\n🤖 Quick Test Responses:")
        print("-" * 40)
        
        for i, question in enumerate(test_questions, 1):
            print(f"\n{i}. 👤 User: {question}")
            
            try:
                response = chatbot.chat(question)
                answer = response['response']
                response_type = response['type']
                
                # Truncate long responses for display
                if len(answer) > 100:
                    display_answer = answer[:100] + "..."
                else:
                    display_answer = answer
                
                icon = "🏥" if response_type == 'healthcare' else "⚠️" if response_type == 'out_of_domain' else "💬"
                print(f"   {icon} Bot: {display_answer}")
                print(f"   📊 Type: {response_type}")
                
            except Exception as e:
                print(f"   ❌ Error: {e}")
        
        print("\n✅ Pre-deployment testing completed!")
        testing_passed = True
        
    except Exception as e:
        print(f"❌ Pre-deployment test failed: {e}")
        print("\n🔧 TROUBLESHOOTING:")
        print("1. Check that model files are complete")
        print("2. Verify model was trained successfully")
        print("3. Try retraining if model is corrupted")
        
        testing_passed = False
        chatbot = None
        
else:
    print("⚠️ Cannot test - no model ready for deployment")
    testing_passed = False
    chatbot = None

## 🌐 Step 3: Web Interface Deployment

Now let's launch your chatbot with a beautiful web interface!

In [ ]:
# Deployment configuration
DEPLOYMENT_CONFIG = {
    'port': 7860,           # Port for web interface
    'share': False,         # Set to True to create public link
    'auth': None,           # Set to ('username', 'password') for authentication
    'title': f'Healthcare Assistant - {MODEL_NAME}',
    'description': 'AI-powered healthcare chatbot trained on medical Q&A data'
}

print("🌐 WEB DEPLOYMENT CONFIGURATION")
print("="*40)
for key, value in DEPLOYMENT_CONFIG.items():
    print(f"{key}: {value}")

print("\n💡 DEPLOYMENT OPTIONS:")
print("-"*25)
print("• Local only: share=False (default)")
print("• Public link: share=True (accessible from internet)")
print("• With auth: auth=('user', 'pass') for password protection")
print("• Custom port: port=8080 (if 7860 is busy)")

# Modify deployment config if needed
# DEPLOYMENT_CONFIG['share'] = True  # Uncomment for public access
# DEPLOYMENT_CONFIG['auth'] = ('admin', 'password123')  # Uncomment for auth

In [ ]:
# Launch web interface
if deployment_ready and testing_passed:
    print("🚀 LAUNCHING WEB INTERFACE")
    print("="*35)
    
    # Check if Gradio is available
    try:
        import gradio as gr
        print("✅ Gradio available")
    except ImportError:
        print("❌ Gradio not installed")
        print("Installing Gradio...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'gradio>=3.35.0'], 
                      capture_output=True)
        import gradio as gr
        print("✅ Gradio installed and imported")
    
    try:
        # Import web interface
        from web_interface import HealthcareChatbotWebInterface
        
        print(f"🔄 Initializing web interface for {MODEL_NAME}...")
        
        # Create web interface
        web_interface = HealthcareChatbotWebInterface(
            model_path=MODEL_PATH,
            share=DEPLOYMENT_CONFIG['share'],
            server_port=DEPLOYMENT_CONFIG['port'],
            auth=DEPLOYMENT_CONFIG['auth']
        )
        
        print("✅ Web interface initialized!")
        print("\n🌐 STARTING WEB SERVER...")
        print("-"*30)
        
        if DEPLOYMENT_CONFIG['share']:
            print("🌍 Creating public link (accessible from anywhere)...")
        else:
            print("🏠 Starting local server...")
        
        print(f"📡 Port: {DEPLOYMENT_CONFIG['port']}")
        
        if DEPLOYMENT_CONFIG['auth']:
            print(f"🔒 Authentication: Enabled")
        else:
            print(f"🔓 Authentication: Disabled")
        
        # Launch interface
        print("\n🚀 Launching... (This may take a moment)")
        
        # Note: In Jupyter, this will block the cell
        # The interface will be available at the displayed URL
        web_interface.launch()
        
    except Exception as e:
        print(f"❌ Error launching web interface: {e}")
        print("\n🔧 TROUBLESHOOTING:")
        print("1. Check if port is already in use")
        print("2. Try a different port number")
        print("3. Restart the notebook kernel")
        print("4. Check firewall settings")
        
        # Alternative: Launch via command line
        print("\n🔄 ALTERNATIVE: Command Line Launch")
        print("-"*40)
        cmd = f"python -m src.web_interface --model_path {MODEL_PATH}"
        if DEPLOYMENT_CONFIG['share']:
            cmd += " --share"
        if DEPLOYMENT_CONFIG['auth']:
            cmd += f" --auth {DEPLOYMENT_CONFIG['auth'][0]} {DEPLOYMENT_CONFIG['auth'][1]}"
        
        print(f"Run this command in terminal:")
        print(f"  {cmd}")
        
else:
    print("⚠️ Cannot launch web interface - prerequisites not met")
    
    if not deployment_ready:
        print("❌ No model ready for deployment")
    if not testing_passed:
        print("❌ Pre-deployment testing failed")
    
    print("\n🔧 SOLUTIONS:")
    print("1. Make sure you have a trained model")
    print("2. Check that pre-deployment testing passes")
    print("3. Verify model files are complete")

## 🖥️ Step 4: Alternative Deployment Methods

If the web interface above doesn't work, here are alternative ways to deploy your chatbot.

In [ ]:
# Alternative deployment methods
if deployment_ready:
    print("🖥️ ALTERNATIVE DEPLOYMENT METHODS")
    print("="*45)
    
    print("1. 💻 Command Line Interface:")
    print("-"*35)
    print(f"   python -m src.chatbot {MODEL_PATH}")
    print("   → Interactive terminal chat")
    print("   → Good for testing and development")
    
    print("\n2. 🌐 Web Interface (Command Line):")
    print("-"*40)
    print(f"   python -m src.web_interface --model_path {MODEL_PATH}")
    print("   → Beautiful web interface")
    print("   → Accessible via browser")
    
    print("\n3. 🌍 Public Web Interface:")
    print("-"*30)
    print(f"   python -m src.web_interface --model_path {MODEL_PATH} --share")
    print("   → Creates public Gradio link")
    print("   → Shareable with anyone")
    
    print("\n4. 🔒 Secured Web Interface:")
    print("-"*30)
    print(f"   python -m src.web_interface --model_path {MODEL_PATH} --auth username password")
    print("   → Password protected")
    print("   → Secure access")
    
    print("\n5. 🐍 Python API:")
    print("-"*20)
    print("   from src.chatbot import HealthcareChatbot")
    print(f"   chatbot = HealthcareChatbot('{MODEL_PATH}')")
    print("   response = chatbot.chat('What are flu symptoms?')")
    print("   → Programmatic access")
    print("   → Integration with other apps")
    
    # Create deployment scripts
    print("\n📝 CREATING DEPLOYMENT SCRIPTS...")
    print("-"*40)
    
    # Script 1: Simple web deployment
    web_script = f"""#!/bin/bash
# Healthcare Chatbot Web Deployment
echo "🏥 Starting Healthcare Chatbot Web Interface..."
echo "Model: {MODEL_NAME}"
echo "Access via: http://localhost:{DEPLOYMENT_CONFIG['port']}"
echo "Press Ctrl+C to stop"
echo ""

python -m src.web_interface --model_path {MODEL_PATH} --port {DEPLOYMENT_CONFIG['port']}
"""
    
    script_path = '/workspace/deploy_web.sh'
    with open(script_path, 'w') as f:
        f.write(web_script)
    os.chmod(script_path, 0o755)
    print(f"✅ Created: {script_path}")
    
    # Script 2: CLI deployment
    cli_script = f"""#!/bin/bash
# Healthcare Chatbot CLI Deployment
echo "🏥 Starting Healthcare Chatbot CLI..."
echo "Model: {MODEL_NAME}"
echo "Type 'quit' to exit"
echo ""

python -m src.chatbot {MODEL_PATH}
"""
    
    cli_script_path = '/workspace/deploy_cli.sh'
    with open(cli_script_path, 'w') as f:
        f.write(cli_script)
    os.chmod(cli_script_path, 0o755)
    print(f"✅ Created: {cli_script_path}")
    
    # Python example script
    python_example = f"""#!/usr/bin/env python3
"""
Healthcare Chatbot Python API Example
Shows how to use the chatbot programmatically.
"""

import sys
import os
sys.path.append('/workspace/src')

from chatbot import HealthcareChatbot

def main():
    print("🏥 Healthcare Chatbot API Example")
    print("="*40)
    
    # Initialize chatbot
    chatbot = HealthcareChatbot('{MODEL_PATH}')
    print(f"✅ Chatbot loaded: {MODEL_NAME}")
    
    # Example questions
    questions = [
        "What are the symptoms of diabetes?",
        "How can I prevent heart disease?",
        "What should I do for a headache?"
    ]
    
    print("\n🤖 Example Conversations:")
    print("-" * 30)
    
    for i, question in enumerate(questions, 1):
        print(f"\n{{i}}. User: {{question}}")
        
        response = chatbot.chat(question)
        print(f"   Bot: {{response['response'][:100]}}...")
        print(f"   Type: {{response['type']}}")
    
    print("\n✅ API example completed!")

if __name__ == "__main__":
    main()
"""
    
    python_example_path = '/workspace/chatbot_api_example.py'
    with open(python_example_path, 'w') as f:
        f.write(python_example)
    os.chmod(python_example_path, 0o755)
    print(f"✅ Created: {python_example_path}")
    
    print("\n📋 USAGE INSTRUCTIONS:")
    print("-"*25)
    print(f"• Web interface: ./deploy_web.sh")
    print(f"• CLI interface: ./deploy_cli.sh")
    print(f"• API example: python chatbot_api_example.py")
    
else:
    print("⚠️ Cannot create deployment alternatives - no model ready")

## 📊 Step 5: Deployment Monitoring and Testing

Let's set up monitoring and testing for your deployed chatbot.

In [ ]:
# Create deployment monitoring
if deployment_ready:
    print("📊 DEPLOYMENT MONITORING SETUP")
    print("="*40)
    
    # Create monitoring script
    monitoring_script = f"""#!/usr/bin/env python3
"""
Healthcare Chatbot Deployment Monitor
Monitors the health and performance of the deployed chatbot.
"""

import sys
import os
import time
import requests
from datetime import datetime
sys.path.append('/workspace/src')

def check_model_health():
    """Check if the model can be loaded and responds correctly."""
    try:
        from chatbot import HealthcareChatbot
        
        chatbot = HealthcareChatbot('{MODEL_PATH}')
        
        # Test basic functionality
        test_response = chatbot.chat("Hello")
        
        return True, "Model is healthy"
    except Exception as e:
        return False, f"Model error: {{e}}"

def check_web_interface(port={DEPLOYMENT_CONFIG['port']}):
    """Check if web interface is accessible."""
    try:
        response = requests.get(f"http://localhost:{{port}}", timeout=5)
        if response.status_code == 200:
            return True, "Web interface is accessible"
        else:
            return False, f"Web interface returned status {{response.status_code}}"
    except Exception as e:
        return False, f"Web interface not accessible: {{e}}"

def main():
    print("🏥 Healthcare Chatbot Monitor")
    print("=" * 40)
    print(f"Model: {MODEL_NAME}")
    print(f"Timestamp: {{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}}")
    print()
    
    # Check model health
    model_ok, model_msg = check_model_health()
    status_icon = "✅" if model_ok else "❌"
    print(f"{{status_icon}} Model Health: {{model_msg}}")
    
    # Check web interface
    web_ok, web_msg = check_web_interface()
    status_icon = "✅" if web_ok else "❌"
    print(f"{{status_icon}} Web Interface: {{web_msg}}")
    
    # Overall status
    overall_ok = model_ok and web_ok
    status = "🟢 HEALTHY" if overall_ok else "🔴 ISSUES DETECTED"
    print(f"\n📊 Overall Status: {{status}}")
    
    return overall_ok

if __name__ == "__main__":
    main()
"""
    
    monitor_path = '/workspace/monitor_deployment.py'
    with open(monitor_path, 'w') as f:
        f.write(monitoring_script)
    os.chmod(monitor_path, 0o755)
    print(f"✅ Created monitoring script: {monitor_path}")
    
    # Create test suite
    test_suite = f"""#!/usr/bin/env python3
"""
Healthcare Chatbot Test Suite
Comprehensive testing for the deployed chatbot.
"""

import sys
import os
sys.path.append('/workspace/src')

from chatbot import HealthcareChatbot

def run_functionality_tests():
    """Test core chatbot functionality."""
    print("🧪 Running Functionality Tests...")
    
    chatbot = HealthcareChatbot('{MODEL_PATH}')
    
    test_cases = [
        ("Hello", "greeting"),
        ("What are diabetes symptoms?", "healthcare"),
        ("What's the weather?", "out_of_domain"),
    ]
    
    passed = 0
    total = len(test_cases)
    
    for question, expected_type in test_cases:
        try:
            response = chatbot.chat(question)
            actual_type = response['type']
            
            if actual_type == expected_type:
                print(f"  ✅ {{question[:30]}}... → {{actual_type}}")
                passed += 1
            else:
                print(f"  ❌ {{question[:30]}}... → Expected: {{expected_type}}, Got: {{actual_type}}")
        except Exception as e:
            print(f"  ❌ {{question[:30]}}... → Error: {{e}}")
    
    print(f"\n📊 Functionality Tests: {{passed}}/{{total}} passed ({{passed/total*100:.0f}}%)")
    return passed == total

def run_performance_tests():
    """Test chatbot performance."""
    print("\n⚡ Running Performance Tests...")
    
    chatbot = HealthcareChatbot('{MODEL_PATH}')
    
    import time
    
    # Response time test
    start_time = time.time()
    response = chatbot.chat("What are the symptoms of diabetes?")
    response_time = time.time() - start_time
    
    print(f"  📏 Response time: {{response_time:.2f}} seconds")
    
    # Response quality test
    response_length = len(response['response'].split())
    print(f"  📝 Response length: {{response_length}} words")
    
    # Performance assessment
    fast_enough = response_time < 10.0  # Should respond within 10 seconds
    good_length = 10 <= response_length <= 200  # Reasonable response length
    
    performance_ok = fast_enough and good_length
    
    if fast_enough:
        print(f"  ✅ Response time acceptable")
    else:
        print(f"  ❌ Response time too slow (>10s)")
    
    if good_length:
        print(f"  ✅ Response length appropriate")
    else:
        print(f"  ❌ Response length inappropriate (<10 or >200 words)")
    
    return performance_ok

def main():
    print("🏥 Healthcare Chatbot Test Suite")
    print("=" * 40)
    print(f"Model: {MODEL_NAME}")
    print()
    
    # Run tests
    functionality_ok = run_functionality_tests()
    performance_ok = run_performance_tests()
    
    # Overall result
    all_passed = functionality_ok and performance_ok
    
    print("\n" + "=" * 40)
    if all_passed:
        print("🎉 ALL TESTS PASSED! Chatbot is ready for deployment.")
    else:
        print("⚠️ Some tests failed. Please review and fix issues.")
    
    return all_passed

if __name__ == "__main__":
    main()
"""
    
    test_path = '/workspace/test_deployment.py'
    with open(test_path, 'w') as f:
        f.write(test_suite)
    os.chmod(test_path, 0o755)
    print(f"✅ Created test suite: {test_path}")
    
    print("\n📋 MONITORING & TESTING:")
    print("-"*30)
    print(f"• Monitor health: python monitor_deployment.py")
    print(f"• Run tests: python test_deployment.py")
    print(f"• Check logs: tail -f /workspace/logs/*.log")
    
else:
    print("⚠️ Cannot create monitoring - no model ready")

## 🚀 Step 6: Production Deployment Guide

Learn about deploying your chatbot for real-world use.

In [ ]:
# Production deployment guide
print("🚀 PRODUCTION DEPLOYMENT GUIDE")
print("="*45)

print("\n🏗️ DEPLOYMENT OPTIONS:")
print("-"*25)

deployment_options = {
    "🌐 Cloud Platforms": {
        "Hugging Face Spaces": "Free hosting for Gradio apps",
        "Google Colab": "Free GPU access with Gradio",
        "AWS EC2": "Scalable cloud instances",
        "Google Cloud": "Managed ML services",
        "Azure": "Enterprise cloud solutions"
    },
    "🐳 Containerization": {
        "Docker": "Containerized deployment",
        "Kubernetes": "Orchestrated scaling",
        "Docker Compose": "Multi-service setup"
    },
    "🖥️ Self-Hosted": {
        "Local Server": "On-premises deployment",
        "VPS": "Virtual private server",
        "Dedicated Server": "Full control setup"
    }
}

for category, options in deployment_options.items():
    print(f"\n{category}:")
    for option, description in options.items():
        print(f"  • {option}: {description}")

print("\n🔧 PRODUCTION CHECKLIST:")
print("-"*30)

checklist = [
    "✅ Model performance validated",
    "✅ Security measures implemented",
    "✅ Error handling robust",
    "✅ Monitoring and logging setup",
    "✅ Backup and recovery plan",
    "✅ Load testing completed",
    "✅ Documentation updated",
    "✅ User feedback mechanism"
]

for item in checklist:
    print(f"  {item}")

print("\n🛡️ SECURITY CONSIDERATIONS:")
print("-"*35)

security_tips = [
    "🔐 Enable authentication for sensitive deployments",
    "🌐 Use HTTPS in production",
    "🚫 Implement rate limiting",
    "📝 Log all interactions for audit",
    "🔍 Monitor for abuse or misuse",
    "⚠️ Add medical disclaimers",
    "🏥 Ensure HIPAA compliance if needed"
]

for tip in security_tips:
    print(f"  {tip}")

print("\n📊 SCALING CONSIDERATIONS:")
print("-"*30)

scaling_tips = [
    "📈 Monitor resource usage (CPU, memory, GPU)",
    "⚡ Implement caching for common queries",
    "🔄 Use load balancers for multiple instances",
    "📦 Consider model optimization (quantization)",
    "🎯 Implement request queuing",
    "📱 Optimize for mobile devices"
]

for tip in scaling_tips:
    print(f"  {tip}")

## 🎯 Step 7: Deployment Summary and Next Steps

Congratulations! Your healthcare chatbot is ready for the world!

In [ ]:
# Create deployment summary
print("🎉 DEPLOYMENT COMPLETE!")
print("="*40)

if deployment_ready:
    print(f"✅ Successfully prepared {MODEL_NAME} for deployment!")
    
    # Create deployment summary
    deployment_summary = {
        'model_info': {
            'name': MODEL_NAME,
            'path': MODEL_PATH,
            'deployment_date': datetime.now().isoformat()
        },
        'deployment_config': DEPLOYMENT_CONFIG,
        'access_methods': {
            'web_interface': f"python -m src.web_interface --model_path {MODEL_PATH}",
            'cli_interface': f"python -m src.chatbot {MODEL_PATH}",
            'public_web': f"python -m src.web_interface --model_path {MODEL_PATH} --share",
            'secured_web': f"python -m src.web_interface --model_path {MODEL_PATH} --auth user pass"
        },
        'monitoring': {
            'health_check': 'python monitor_deployment.py',
            'test_suite': 'python test_deployment.py'
        },
        'files_created': [
            '/workspace/deploy_web.sh',
            '/workspace/deploy_cli.sh',
            '/workspace/chatbot_api_example.py',
            '/workspace/monitor_deployment.py',
            '/workspace/test_deployment.py'
        ]
    }
    
    # Save deployment summary
    summary_path = '/workspace/notebooks/deployment_summary.json'
    try:
        with open(summary_path, 'w') as f:
            json.dump(deployment_summary, f, indent=2)
        print(f"💾 Deployment summary saved to: {summary_path}")
    except Exception as e:
        print(f"⚠️ Could not save summary: {e}")
    
    print("\n🚀 QUICK START COMMANDS:")
    print("-"*30)
    print(f"🌐 Web Interface:")
    print(f"   ./deploy_web.sh")
    print(f"   OR: python -m src.web_interface --model_path {MODEL_PATH}")
    
    print(f"\n💻 CLI Interface:")
    print(f"   ./deploy_cli.sh")
    print(f"   OR: python -m src.chatbot {MODEL_PATH}")
    
    print(f"\n🌍 Public Access:")
    print(f"   python -m src.web_interface --model_path {MODEL_PATH} --share")
    
    print("\n📊 MONITORING & TESTING:")
    print("-"*30)
    print("🔍 Health Check: python monitor_deployment.py")
    print("🧪 Run Tests: python test_deployment.py")
    
    print("\n🎯 WHAT YOUR USERS GET:")
    print("-"*25)
    print("✅ AI-powered healthcare assistance")
    print("✅ Natural language conversation")
    print("✅ Medical knowledge and advice")
    print("✅ Out-of-domain query handling")
    print("✅ Beautiful web interface")
    print("✅ Mobile-friendly design")
    print("✅ Conversation export")
    print("✅ Health tips and guidance")
    
else:
    print("❌ Deployment preparation failed.")
    print("\n🔧 TROUBLESHOOTING:")
    print("-"*25)
    print("1. Make sure you have a trained model")
    print("2. Check that training completed successfully")
    print("3. Verify model files are complete")
    print("4. Try retraining if model is corrupted")

print("\n🏆 ACHIEVEMENT UNLOCKED:")
print("-"*30)
print("🎓 You've successfully built and deployed an AI healthcare chatbot!")
print("🏥 Your AI doctor is ready to help patients worldwide!")
print("🤖 From data to deployment - you've mastered the full ML pipeline!")

print("\n📚 CONTINUE LEARNING:")
print("-"*25)
print("• Experiment with different models and datasets")
print("• Add new features like voice interaction")
print("• Integrate with real healthcare systems")
print("• Explore other medical AI applications")
print("• Share your work with the community")

print("\n🌟 Thank you for building responsible AI for healthcare! 🌟")